In [1]:
# fix imports
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import torch
from notebooks.experiment_robust import load_robust_experiment
from notebooks.experiment_torch import load_torchvision_experiment

model, dl_train, dl_eval = load_robust_experiment("Standard_R50", "imagenet")
# model, dl_train, dl_eval = load_torchvision_experiment("alexnet", batch_size=32)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /home/fre.gilad/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:03<00:00, 64.5MB/s] 


Creating Dataset:   0%|          | 0/10000 [00:00<?, ?it/s]

Creating Dataset:   0%|          | 0/5000 [00:00<?, ?it/s]

Extracting Samples:   0%|          | 0/10 [00:00<?, ?it/s]

Extracting Samples:   0%|          | 0/5 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/313 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/157 [00:00<?, ?it/s]

Model           : AlexNet
Device          : cuda
Clean Eval Acc  : 56.72
Clean Train Acc : 73.98
Weight          : IMAGENET1K_V1
Metrics         : {'ImageNet-1K': {'acc@1': 56.522, 'acc@5': 79.066}}


In [ ]:
from ulib.attack import StopCriteria
from ulib.attacks.usgd import USGD
from ulib.pert_module import PertModule
from ulib import eval

stop = StopCriteria(max_epochs=10, max_time=60 * 10, patience=5, patience_delta=0.001)
pert_model = PertModule(model, data_shape=dl_train.get_tensor(0).shape[1:], eps=8 / 255, random_init=True)
optimizer = torch.optim.Adam(pert_model.parameters(), lr=1e-2, maximize=True)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=100, T_mult=1)
grad_scaler = torch.GradScaler(device=pert_model.device.type)

attack = USGD(
    pert_model=pert_model,
    optimizer=optimizer,
    criterion=criterion,
    skip_already_fooled=False,
    scheduler=scheduler,
    grad_scaler=grad_scaler,
    sched_on_batch=True,
    eval_on_batch=True,
    eval_freq=100,
    targeted=False,
    metric_func=eval.attack_success_ratio,
    # log_dir="logs",
)

pert = attack.fit(dl_train, dl_eval, stop)

In [ ]:
attack.close()

In [ ]:
from ulib import eval

eval.full_analysis(pert_model, dl_eval)